In [51]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
import matplotlib.pyplot as plt

In [40]:
spark = SparkSession.builder.appName("myapp").master('local[*]').getOrCreate()

In [42]:
df = spark.read.csv('dataset/Base.csv', header=True, inferSchema=True)

Here i'm checking how many cores are being used by spark

In [48]:
num_cores = spark.sparkContext.defaultParallelism
print(f"Spark is using {num_cores} cores.")

Spark is using 12 cores.


In [43]:
df.show()

24/11/06 16:44:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------------+---------------------+-------------------------+----------------------------+------------+------------------+----------------------+------------+------------+------------------+------------------+------------------+--------------------+--------------------------------+-----------------+-----------------+-------------+--------------+----------------+------------------+-----------------+---------------+---------------------+---------------+--------+-------------------------+---------+------------------+-------------------------+------------------+-----+
|fraud_bool|            income|name_email_similarity|prev_address_months_count|current_address_months_count|customer_age|days_since_request|intended_balcon_amount|payment_type|zip_count_4w|       velocity_6h|      velocity_24h|       velocity_4w|bank_branch_count_8w|date_of_birth_distinct_emails_4w|employment_status|credit_risk_score|email_is_free|housing_status|phone_home_valid|phone_mobile_valid|bank_month

In [47]:
#Check if there are any missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+------+---------------------+-------------------------+----------------------------+------------+------------------+----------------------+------------+------------+-----------+------------+-----------+--------------------+--------------------------------+-----------------+-----------------+-------------+--------------+----------------+------------------+-----------------+---------------+---------------------+---------------+------+-------------------------+---------+------------------+-------------------------+------------------+-----+
|fraud_bool|income|name_email_similarity|prev_address_months_count|current_address_months_count|customer_age|days_since_request|intended_balcon_amount|payment_type|zip_count_4w|velocity_6h|velocity_24h|velocity_4w|bank_branch_count_8w|date_of_birth_distinct_emails_4w|employment_status|credit_risk_score|email_is_free|housing_status|phone_home_valid|phone_mobile_valid|bank_months_count|has_other_cards|proposed_credit_limit|foreign_request|sour

There are no missing values in the dataset

In [49]:
#Check for outliers
df.describe().show()

+-------+-------------------+-------------------+---------------------+-------------------------+----------------------------+------------------+--------------------+----------------------+------------+------------------+-------------------+------------------+-----------------+--------------------+--------------------------------+-----------------+-----------------+------------------+--------------+-------------------+-------------------+------------------+-----------------+---------------------+-------------------+--------+-------------------------+---------+-------------------+-------------------------+------------------+------------------+
|summary|         fraud_bool|             income|name_email_similarity|prev_address_months_count|current_address_months_count|      customer_age|  days_since_request|intended_balcon_amount|payment_type|      zip_count_4w|        velocity_6h|      velocity_24h|      velocity_4w|bank_branch_count_8w|date_of_birth_distinct_emails_4w|employment_status

In [54]:
# Convert PySpark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Plot boxplots for each numerical column in the DataFrame
pandas_df.plot(kind='box', subplots=True, layout=(len(pandas_df.columns) // 3 + 1, 3), figsize=(150, 50), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [56]:
#Distribution of the target variable 'fraud_bool'
df.groupBy('fraud_bool').count().show()

#Print the distribution in percentage
df.groupBy('fraud_bool').count().withColumn('percentage', col('count') / df.count() * 100).show()

+----------+------+
|fraud_bool| count|
+----------+------+
|         1| 11029|
|         0|988971|
+----------+------+


+----------+------+-----------------+
|fraud_bool| count|       percentage|
+----------+------+-----------------+
|         1| 11029|           1.1029|
|         0|988971|98.89710000000001|
+----------+------+-----------------+
